## HuggingFace ViT (currently subset)

In [15]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor

# Define dataset transforms
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset and create a subset
train_dataset = CIFAR10(root='./data', train=True, download=False, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=False, transform=transform)

train_subset = Subset(train_dataset, range(100))  # use first 100 examples for training
test_subset = Subset(test_dataset, range(50))  # use first 50 examples for testing

# Define data loaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

# Load pre-trained model and feature extractor
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

# Replace the classification head with a new one
num_classes = len(train_subset.dataset.classes)
model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)

# Train the model on the CIFAR-10 dataset subset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(10):
    train_loss = 0
    train_acc = 0
    test_loss = 0
    test_acc = 0
    
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        logits = outputs.logits  # extract logits from ImageClassifierOutput
        loss = torch.nn.CrossEntropyLoss()(logits, labels)  # pass only logits to CrossEntropyLoss
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        train_acc += (logits.argmax(dim=1) == labels).sum().item()
        
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            logits = outputs.logits  # extract logits from ImageClassifierOutput
            loss = torch.nn.CrossEntropyLoss()(logits, labels)  # pass only logits to CrossEntropyLoss
            
            test_loss += loss.item() * images.size(0)
            test_acc += (logits.argmax(dim=1) == labels).sum().item()
            
    train_loss /= len(train_subset)
    train_acc /= len(train_subset)
    test_loss /= len(test_subset)
    test_acc /= len(test_subset)
    
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')


Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Train Loss: 2.1428, Train Acc: 0.2100, Test Loss: 1.9686, Test Acc: 0.2800
Epoch 2, Train Loss: 0.8930, Train Acc: 0.9300, Test Loss: 1.4578, Test Acc: 0.5800
Epoch 3, Train Loss: 0.4146, Train Acc: 1.0000, Test Loss: 1.1451, Test Acc: 0.7800
Epoch 4, Train Loss: 0.1732, Train Acc: 1.0000, Test Loss: 0.9423, Test Acc: 0.8200
Epoch 5, Train Loss: 0.0854, Train Acc: 1.0000, Test Loss: 0.8350, Test Acc: 0.8200
Epoch 6, Train Loss: 0.0475, Train Acc: 1.0000, Test Loss: 0.7666, Test Acc: 0.8600
Epoch 7, Train Loss: 0.0288, Train Acc: 1.0000, Test Loss: 0.7179, Test Acc: 0.8800
Epoch 8, Train Loss: 0.0194, Train Acc: 1.0000, Test Loss: 0.6811, Test Acc: 0.9000
Epoch 9, Train Loss: 0.0140, Train Acc: 1.0000, Test Loss: 0.6545, Test Acc: 0.9000
Epoch 10, Train Loss: 0.0108, Train Acc: 1.0000, Test Loss: 0.6335, Test Acc: 0.9000


## TorchVision resnet18 (currently subset)

In [18]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torchvision.models import resnet18

# Define dataset transforms
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
train_dataset = CIFAR10(root='./data', train=True, download=False, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=False, transform=transform)

train_subset = Subset(train_dataset, range(100))  # use first 100 examples for training
test_subset = Subset(test_dataset, range(50))  # use first 50 examples for testing

# Define data loaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

# Define data loaders
#train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Load pre-trained model
model = resnet18(pretrained=True)

# Replace the classification head with a new one
num_classes = len(train_subset.dataset.classes)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Train the model on the CIFAR-10 dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(10):
    train_loss = 0
    train_acc = 0
    test_loss = 0
    test_acc = 0
    
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = torch.nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        train_acc += (outputs.argmax(dim=1) == labels).sum().item()
        
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = torch.nn.CrossEntropyLoss()(outputs, labels)
            
            test_loss += loss.item() * images.size(0)
            test_acc += (outputs.argmax(dim=1) == labels).sum().item()
            
    train_loss /= len(train_dataset)
    train_acc /= len(train_dataset)
    test_loss /= len(test_dataset)
    test_acc /= len(test_dataset)
    
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')


Epoch 1, Train Loss: 0.0050, Train Acc: 0.0002, Test Loss: 0.0110, Test Acc: 0.0011
Epoch 2, Train Loss: 0.0025, Train Acc: 0.0014, Test Loss: 0.0102, Test Acc: 0.0012
Epoch 3, Train Loss: 0.0013, Train Acc: 0.0019, Test Loss: 0.0097, Test Acc: 0.0016
Epoch 4, Train Loss: 0.0008, Train Acc: 0.0020, Test Loss: 0.0094, Test Acc: 0.0016
Epoch 5, Train Loss: 0.0005, Train Acc: 0.0020, Test Loss: 0.0090, Test Acc: 0.0017
Epoch 6, Train Loss: 0.0004, Train Acc: 0.0020, Test Loss: 0.0088, Test Acc: 0.0018
Epoch 7, Train Loss: 0.0003, Train Acc: 0.0020, Test Loss: 0.0086, Test Acc: 0.0019
Epoch 8, Train Loss: 0.0002, Train Acc: 0.0020, Test Loss: 0.0085, Test Acc: 0.0020
Epoch 9, Train Loss: 0.0002, Train Acc: 0.0020, Test Loss: 0.0083, Test Acc: 0.0021
Epoch 10, Train Loss: 0.0001, Train Acc: 0.0020, Test Loss: 0.0080, Test Acc: 0.0021
